In [1]:
import pandas as pd
import numpy as np

from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="new_access_ctrl_v2",timeout=200)

import pymysql

con = pymysql.connect('localhost', 'root', 'HIDDEN', 'HIDDEN')

In [2]:
#fetch the cities tables from the w3 databases
def select_main():
    con.connect()
    select = con.cursor()
    statement = 'select distinct city,country from customers;'
    select.execute(statement)
    data = select.fetchall()
    con.commit()
    
    cities = pd.DataFrame(data).rename({0:'city',1:'country'},axis=1)
    
    return cities

In [3]:
#load into memory
cities = select_main()

In [4]:
#append the latitude, longitude into lists, merge them into the DataFrame, and return it
def get_points(cities):
    
    lat = []
    lon = []

    for city, country in cities.values:
        string = city +', '+ country
        coords = geolocator.geocode(string, language='en')
        lat.append(coords.latitude)
        lon.append(coords.longitude)
    
    cities['latitude'] = lat
    cities['longitude'] = lon
    
        
    return cities

In [5]:
#new DataFrame with geographic attributes, might take a minute or two
with_points = get_points(cities)

In [9]:
#load into the customly created city_points table, linked to the customer table in the schema. commit the insert
def insert_points(with_points):
    con.connect()
    insert_main = con.cursor()

    for row in with_points[['city','latitude','longitude']].values:
        insert_main.callproc('w3_insert_points',(row[0],row[1],row[2]))

    con.commit()

In [10]:
#
insert_points(with_points)